# Extracting Uber Eats categories' links

Importing libraries

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import time
import json
import csv
import os
import pandas as pd
from urllib.error import URLError
import time
import requests
HEADERS = {'User-Agent': 'Mozilla/5.0', 'Accept': 'text/html,application/xhtml+xml,application/xml'}


Requesting url

In [2]:
url = 'https://www.ubereats.com/gb/category/london-eng'
url = requests.get(url)

Creating beautiful soup object

In [3]:
#Creating beautiful soup object
soup = BeautifulSoup(url.content, 'html.parser')

Getting a list of all category links

In [4]:
#Find category elements
cat_urls = soup.find_all('main', class_='dr')
cat_urls_list = []

#extract link of each category and append it to a list "cat_urls_list"
for ind_cat in cat_urls:
    for link in ind_cat.find_all('a', href=True):
        href = link['href']
        cat_urls_list.append('https://www.ubereats.com'+href)


Defining a function for getting the amount of restaurants in each category

In [5]:
cat_rest_urls_list = []

def get_category_rests(category_url, HEADERS):
    req = Request(category_url, headers=HEADERS)
    webpage = urlopen(req, timeout=10).read()
    rest_soup = BeautifulSoup(webpage, 'html.parser')
    cat_rest_urls = rest_soup.find_all("div", class_= "ak cx") 
    for ind_url in cat_rest_urls:
        for url in ind_url.find_all('a', href=True):
            href = url['href']
            cat_rest_urls_list.append(href)
    return cat_rest_urls_list


Applying function to the urls of my list

In [6]:
# Create an empty dictionary to store the results
category_rests_dict = {}

# Loop through the category urls and call the modified function
for i in cat_urls_list:
    category_rest_urls_list = get_category_rests(i, HEADERS)
    #Get the number of restaurants in each category
    category_rests_dict[i] = len(category_rest_urls_list)

KeyboardInterrupt: 

Create a list of category names starting from the url list

In [8]:
#Split links and store what is after last "/"
london_cat_all = [url.split('/')[-1] for url in cat_urls_list]

Transform the complete dictionary into a dataframe and store it as csv

In [10]:


london_num_rest_cat = pd.DataFrame.from_dict(category_rests_dict, orient='index', columns=['rest_num'])
london_num_rest_cat.index.name = 'url'

#add the list containing categories names as a new column in the dataframe
london_num_rest_cat['category'] = london_cat_all

#save as csv
london_num_rest_cat.to_csv('../../Uber-Eats-London-Scraping/scraped-data/1-London-cat-all.csv', index=True)

At this point, we have a dataset containing the categories for London, the respective links and the amount of restaurants inside each one. Now, lets get the urls of all restaurants inside each category. Move to the "London-restaurant-urls" notebook